In [19]:
import functools
from dynesty import NestedSampler
from dynesty import utils as dyfunc
import numpy as np


def prior_transform(u):
    return u

def integrate_likelihood(x, *args, logp): #x = None, y = None
    if len(args) != 0 or type(x).__name__ == 'float': #and len(x) != 0:        
        X = np.array([x, *args])
        X = np.expand_dims(X, -2)
        output = np.exp(logp(X))[0]
    elif len(args) == 0: #and len([x]) != 0:
        x = np.array(x)
        #print(x)
        output = logp(x).squeeze(-1)
    else:
        print("Number of args: ", len(args))
        print("Args: ", args)
    return output

def dynesty_true_integral(bounds, ndim, dlogz, logp, prior_fac):
    ### Dynesty ###
    ptform_kwargs={'bounds': bounds}
    integrate_likelihood_dy = functools.partial(integrate_likelihood, logp=logp)
    sampler = NestedSampler(integrate_likelihood_dy, prior_transform,
                       nlive=1000, ndim=ndim, 
                       blob=False )#, pool=pool, queue_size=nthreads)
    sampler.run_nested(print_progress=True, dlogz=dlogz)
    
    res = sampler.results
    logz_dy = res.logz[-1] + 2*prior_fac #ln(z_dynesty) + ln(prior_fac) = z_dynesty*prior_fac = Evidence -> Why do we have to do 2xprior_fac?
    logzerr_dy = res.logzerr[-1]
    computed_integrals = dyfunc.compute_integrals(res.logl, res.logvol)
    #log.info(computed_integrals[1][-1], computed_integrals[2][-1])
    print(f"Logz from Dynesty = {logz_dy} +- {logzerr_dy} with {len(res.ncall)} samples")
    return logz_dy, logzerr_dy

In [32]:
ndim = 9
def gaussian(X):
    # X is a N x DIM shaped tensor, output is N tensor
    mean = np.array(0.5) #len(param_list)*
    sigma = np.array(0.1) #len(param_list)*
    return np.expand_dims((-0.5*np.sum((X-mean)**2/sigma**2, axis=-1, keepdims=False)), -1)


dynesty_true_integral([0, 1]*ndim, ndim, 0.1, gaussian, 0)

14369it [00:11, 1213.09it/s, +1000 | bound: 19 | nc: 1 | ncall: 63502 | eff(%): 24.590 | loglstar:   -inf < -0.340 <    inf | logz: -12.409 +/-  0.096 | dlogz:  0.000 >  0.100]

Logz from Dynesty = -12.408803113960568 +- 0.11372623993865191 with 15369 samples


(-12.408803113960568, 0.11372623993865191)